In [1]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage
import tiktoken

In [2]:
examples = [
    {"input": "Gollum", "output": "<Story involving Gollum>"},
    {"input": "Gandalf", "output": "<Story involving Gandalf>"},
    {"input": "Bilbo", "output": "<Story involving Bilbo>"},
]

In [3]:
story_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Character: {input}\nStory: {output}",
)

In [4]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=story_prompt,
    max_length=1000, # 1000 tokens are to be included from examples in the prompt
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
    # You have modified the get_text_length function to work with the TikToken library based on token usage:
    get_text_length=num_tokens_from_string,
)

In [6]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=story_prompt,
    prefix="Generate a story for {character} using the current Character/Story pairs from all of the characters as context.",
    suffix="Character: {character}\nStory:",
    input_variables=["character"],
)

In [7]:
# Provide a new character from lord of the rings
formatted_prompt = dynamic_prompt.format(character="Frodo")

In [8]:
print(formatted_prompt)

Generate a story for Frodo using the current Character/Story pairs from all of the characters as context.

Character: Gollum
Story: <Story involving Gollum>

Character: Gandalf
Story: <Story involving Gandalf>

Character: Bilbo
Story: <Story involving Bilbo>

Character: Frodo
Story:


In [9]:
model = ChatOpenAI()

result = model.invoke([SystemMessage(content=formatted_prompt)])

print(result.content)

Once upon a time, in the peaceful land of Middle-earth, there lived a young hobbit named Frodo Baggins. Frodo hailed from the Shire, a cozy and tranquil place where life revolved around good food, cheerful gatherings, and a love for simple pleasures. Little did Frodo know that his life was about to take a dramatic turn, plunging him into a perilous adventure that would shape the fate of Middle-earth.

It all began when Frodo's beloved uncle, Bilbo Baggins, embarked on a grand journey with a group of dwarves led by Thorin Oakenshield. They sought to reclaim their homeland from the treacherous dragon Smaug. During their quest, Bilbo stumbled upon a mysterious ring that possessed immense power. Unbeknownst to him, this was the One Ring, the ultimate weapon of the Dark Lord Sauron.

Years later, Gandalf the Grey, a wise and powerful wizard, arrived at Frodo's doorstep with grave news. He revealed the true nature of the ring to Frodo, urging him to undertake a perilous mission. The ring had